<a href="https://colab.research.google.com/github/javed163/Recommendation-Systems-Methods/blob/main/surprise_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505166 sha256=96ade9b0702800474204a530678ab5a33b391a4ddc7772ab11a4d99a0df79a92
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [27]:
from surprise import Dataset, Reader
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise.model_selection import cross_validate
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.model_selection import train_test_split

# **Load and preprocess the data**

In [6]:
import pandas as pd
import numpy as np
rating_df = pd.read_csv("/content/drive/MyDrive/Recommendation System/Datasets/ratings.csv")
movies_df = pd.read_csv("/content/drive/MyDrive/Recommendation System/Datasets/movies.csv")

In [7]:
movies_df.head(2)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy


In [11]:
df = pd.merge(rating_df, movies_df[['movieId','genres']], on='movieId', how='left')
df

,userId,movieId,rating,timestamp,genres
0,1,31,2.5,1260759144,Drama
1,1,1029,3.0,1260759179,Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Thriller
3,1,1129,2.0,1260759185,Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Drama
...,...,...,...,...,...
99999,671,6268,2.5,1065579370,Comedy|Drama|Romance
100000,671,6269,4.0,1065149201,Documentary
100001,671,6365,4.0,1070940363,Action|Adventure|Sci-Fi|Thriller|IMAX
100002,671,6385,2.5,1070979663,Drama


In [40]:
import pandas as pd
import numpy as np
rating_df = pd.read_csv("/content/drive/MyDrive/Recommendation System/Datasets/ratings.csv")
movies_df = pd.read_csv("/content/drive/MyDrive/Recommendation System/Datasets/movies.csv")

# Merge DataFrames ensuring 'genres' column is included
df = pd.merge(rating_df, movies_df[['movieId', 'genres']], on='movieId', how='left')

# Initialize encoders and binarizer
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()
mlb = MultiLabelBinarizer()

# Make a copy of the DataFrame
df = df.copy()

# Encode user and movie IDs
df['user_id'] = user_encoder.fit_transform(df['userId'])
df['movie_id'] = movie_encoder.fit_transform(df['movieId'])

# Check if 'genres' column exists before proceeding
if 'genres' in df.columns:
    # Apply one-hot encoding to the 'genres' column
    genres_encoded = pd.DataFrame(mlb.fit_transform(df['genres'].str.split('|')),
                                  columns=mlb.classes_,
                                  index=df.index)

    # Join encoded genres with the DataFrame
    df = df.join(genres_encoded)
else:
    print("Warning: 'genres' column not found in the DataFrame. Skipping genre encoding.")

In [41]:
df

,userId,movieId,rating,timestamp,genres,user_id,movie_id,(no genres listed),Action,Adventure,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,31,2.5,1260759144,Drama,0,30,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1029,3.0,1260759179,Animation|Children|Drama|Musical,0,833,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,1,1061,3.0,1260759182,Thriller,0,859,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1,1129,2.0,1260759185,Action|Adventure|Sci-Fi|Thriller,0,906,0,1,1,...,0,0,0,0,0,0,1,1,0,0
4,1,1172,4.0,1260759205,Drama,0,931,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99999,671,6268,2.5,1065579370,Comedy|Drama|Romance,670,4545,0,0,0,...,0,0,0,0,0,1,0,0,0,0
100000,671,6269,4.0,1065149201,Documentary,670,4546,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100001,671,6365,4.0,1070940363,Action|Adventure|Sci-Fi|Thriller|IMAX,670,4597,0,1,1,...,0,0,1,0,0,0,1,1,0,0
100002,671,6385,2.5,1070979663,Drama,670,4610,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
df.drop(columns = ['genres','(no genres listed)'], inplace  = True)

In [43]:
df

,userId,movieId,rating,timestamp,user_id,movie_id,Action,Adventure,Animation,Children,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,31,2.5,1260759144,0,30,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1029,3.0,1260759179,0,833,0,0,1,1,...,0,0,0,1,0,0,0,0,0,0
2,1,1061,3.0,1260759182,0,859,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1,1129,2.0,1260759185,0,906,1,1,0,0,...,0,0,0,0,0,0,1,1,0,0
4,1,1172,4.0,1260759205,0,931,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99999,671,6268,2.5,1065579370,670,4545,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
100000,671,6269,4.0,1065149201,670,4546,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100001,671,6365,4.0,1070940363,670,4597,1,1,0,0,...,0,0,1,0,0,0,1,1,0,0
100002,671,6385,2.5,1070979663,670,4610,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Buid the Model with collaborative filtering

In [44]:
train_df, test_df = train_test_split(df, test_size = 0.2)
train_df

,userId,movieId,rating,timestamp,user_id,movie_id,Action,Adventure,Animation,Children,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
2046,15,7003,2.0,1367800233,14,4916,0,0,0,0,...,0,0,0,0,1,0,1,1,0,0
35374,253,3745,2.0,1215823624,252,2997,1,1,1,1,...,0,0,0,0,0,0,1,0,0,0
79280,547,3430,0.5,1053173832,546,2736,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
65494,466,1688,5.0,944979201,465,1341,0,1,1,1,...,0,0,0,1,0,0,0,0,0,0
98704,662,509,3.0,839022731,661,455,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29433,213,1566,4.0,1462638424,212,1246,0,1,1,1,...,0,0,0,1,0,0,0,0,0,0
91702,607,4963,4.5,1113319563,606,3845,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
44673,313,587,3.0,1168878664,312,520,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
66089,468,2161,3.0,1296197050,467,1716,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [45]:
reader = Reader(rating_scale = (0.5,5))
data = Dataset.load_from_df(train_df[['userId','movieId','rating']],reader)
trainset = data.build_full_trainset()

In [46]:
model_svd = SVD()
model_svd.fit(trainset)

# Correct the method name to build_anti_testset
prediction_svd = model_svd.test(trainset.build_anti_testset())

# Import accuracy module from surprise
from surprise import accuracy
accuracy.rmse(prediction_svd)

RMSE: 0.4697


0.4697297948191685

# Make Recommendations

In [54]:
# collaborative filtering
def get_recommendations_cf(user_id, top_n=10):
    user_movies = df[df['userId'] == user_id]['movieId'].unique()
    all_movies = df['movieId'].unique()

    movie_to_predict = list(set(all_movies) - set(user_movies))[:top_n]

    user_movie_pairs = [ (user_id, movie_id, 0) for movie_id in movie_to_predict]
    prediction_df = model_svd.test(user_movie_pairs)

    top_n_recommendation = sorted(prediction_df, key=lambda x: x.est, reverse=True)[:top_n]
    top_n_movies_ids = [int(pred.iid) for pred in top_n_recommendation]
    top_n_movies = movie_encoder.inverse_transform(top_n_movies_ids)

    return top_n_movies

In [57]:

user_id = 55
recommendations = get_recommendations_cf(user_id)
top_n_movies_title = movies_df[movies_df['movieId'].isin(recommendations)]['title'].tolist()

print(f"Recommended movies for user {user_id}:")
for i,title in enumerate(top_n_movies_title,1):
    print(f"{i}. {title}")


Recommended movies for user 55:
1. Grumpier Old Men (1995)
2. Waiting to Exhale (1995)
3. Father of the Bride Part II (1995)
4. Sudden Death (1995)
5. American President, The (1995)
6. Dracula: Dead and Loving It (1995)
7. Balto (1995)
8. Nixon (1995)
9. Cutthroat Island (1995)
10. Casino (1995)
